In [ ]:
import os
import pandas as pd
import numpy as np



In [127]:
drivers_df = pd.read_csv('F1KaggleData/drivers.csv')
races_df = pd.read_csv('F1KaggleData/races.csv')
circuits_df = pd.read_csv('F1KaggleData/circuits.csv')
results_df = pd.read_csv('F1KaggleData/results.csv')


In [ ]:
display(drivers_df.head(5))
display(races_df.head(5))
display(circuits_df.head(5))
display(results_df.head(5))


In [ ]:
display('Total Number of Circuits used in F1: {}'.format(circuits_df['circuitId'].max()))
display('Total Drivers in F1: {}'.format(drivers_df['driverId'].max()))


Drivers with a win?  

How do we work this out... 

In [ ]:
wins_df = results_df.loc[results_df['position'] == 1.0]

display(wins_df.sample(10))

drivers_with_wins = wins_df['driverId'].unique()

display(drivers_with_wins)

display('{} drivers in F1 have won a race'.format(len(drivers_with_wins)))

names_of_drivers_with_wins = drivers_df[['driverRef', 'nationality']].loc[drivers_df['driverId'].isin(drivers_with_wins)]

display(names_of_drivers_with_wins)

In [ ]:
# can we get a pie chart of the nationality breakdown

winners_by_nationality = names_of_drivers_with_wins['nationality'].value_counts()

display(winners_by_nationality)

names_of_drivers_with_wins['nationality'].value_counts().plot.pie(subplots=True)




In [86]:
# ok, lets try winners by decade here. 

# however lets do this via some dictionary comprehension 

# pandas first
display(results_df.head(5))
# display(races_df.head(5))

first_race = races_df['date'].min()
display('Date of First Race in dataset: {}'.format(first_race))
last_race = races_df['date'].max()
display('Date of Last Race in dataset: {}'.format(last_race))

# sort it becuase this current format hurts my brain 
races_df.sort_values(by=['date'], inplace=True)
display(races_df.head(5))
list_of_dates = races_df['date'].values
# display(list_of_dates)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1


'Date of First Race in dataset: 1950-05-13'

'Date of Last Race in dataset: 2018-11-25'

,raceId,year,round,circuitId,name,date,time,url
832,833,1950,1,9,British Grand Prix,1950-05-13,,http://en.wikipedia.org/wiki/1950_British_Gran...
833,834,1950,2,6,Monaco Grand Prix,1950-05-21,,http://en.wikipedia.org/wiki/1950_Monaco_Grand...
834,835,1950,3,19,Indianapolis 500,1950-05-30,,http://en.wikipedia.org/wiki/1950_Indianapolis...
835,836,1950,4,66,Swiss Grand Prix,1950-06-04,,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...
836,837,1950,5,13,Belgian Grand Prix,1950-06-18,,http://en.wikipedia.org/wiki/1950_Belgian_Gran...


In [ ]:
# display(results_df.head(10))

winner_of_each_race_df = results_df.loc[results_df['position'] == 1.0]
winners = set(winner_of_each_race_df['driverId'].to_list())
display(winner_of_each_race_df.head(5))
count_of_wins_per_driver = winner_of_each_race_df['driverId'].value_counts()
display(count_of_wins_per_driver)
display(winners)

In [ ]:
# now lets join the data from the results df and the drivers_df 

winner_code = drivers_df.loc[drivers_df['driverId'].isin(winner_of_each_race_df['driverId'].unique())][['driverId','driverRef']]

# display(winner_of_each_race_df['driverId'].unique())
display(winner_code)

In [91]:
comb = pd.merge(winner_code, winner_of_each_race_df)

comb2 = pd.merge(winner_code, winner_of_each_race_df, on='driverId')

display(comb2)
x = comb.driverRef.value_counts()

display(x)
display(type(x))

,driverId,driverRef,resultId,raceId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,hamilton,1,18,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,1,hamilton,109,23,1,22.0,3,1.0,1,1,10.0,76,00:42.7,7242742.0,71.0,6.0,01:18.5,153.152,1
2,1,hamilton,169,26,1,22.0,4,1.0,1,1,10.0,60,39:09.4,5949440.0,16.0,3.0,01:32.8,199.398,1
3,1,hamilton,189,27,1,22.0,1,1.0,1,1,10.0,67,31:20.9,5480874.0,17.0,2.0,01:16.0,216.552,1
4,1,hamilton,329,34,1,22.0,1,1.0,1,1,10.0,56,31:57.4,5517403.0,13.0,1.0,01:36.3,203.722,1
5,1,hamilton,479,41,1,2.0,1,1.0,1,1,10.0,70,44:11.3,6251292.0,37.0,2.0,01:16.5,205.239,1
6,1,hamilton,501,42,1,2.0,1,1.0,1,1,10.0,73,31:10.0,5469965.0,20.0,2.0,01:13.2,206.101,1
7,1,hamilton,589,46,1,2.0,1,1.0,1,1,10.0,70,35:53.0,5752991.0,13.0,2.0,01:20.2,196.724,1
8,1,hamilton,677,50,1,2.0,1,1.0,1,1,10.0,67,00:34.6,7234579.0,27.0,1.0,01:28.2,186.259,1
9,1,hamilton,7734,10,1,1.0,4,1.0,1,1,10.0,70,38:23.9,5903876.0,16.0,4.0,01:22.5,191.219,1


michael_schumacher    91
hamilton              62
prost                 51
vettel                47
senna                 41
alonso                32
mansell               31
stewart               27
lauda                 25
clark                 25
fangio                24
piquet                23
rosberg               23
damon_hill            22
hakkinen              20
raikkonen             20
moss                  16
button                15
jack_brabham          14
hill                  14
emerson_fittipaldi    14
ascari                13
coulthard             13
jones                 12
mario_andretti        12
reutemann             12
barrichello           11
villeneuve            11
massa                 11
hunt                  10
                      ..
wallard                1
bonnier                1
beltoise               1
ireland                1
rathmann               1
bandini                1
parsons                1
kubica                 1
ginther                1


pandas.core.series.Series

In [121]:
# Select to see wins by driver drop down 

import ipywidgets as widgets
from IPython.display import clear_output

f1_winner_dropdown = widgets.Dropdown(
                                    options=winner_code['driverRef'].to_list(),
                                    value=None,
                                    description='Select F1 Winner: ',
                                    disabled=False,
                                    )



# now lets restrict the results purely to them 

display(f1_winner_dropdown)

out = widgets.Output()
display(out)



f1_winner_dropdown.observe(f1_winner_dropdown_eventhandler, names='value')

def f1_winner_dropdown_eventhandler(change):

    with out:
        clear_output()
        selected_driver_id = int(winner_code.loc[winner_code['driverRef'] == change.new]['driverId'])
        display(selected_driver_id)
        display(results_df.loc[(results_df['driverId']== selected_driver_id) & (results_df['position'] == 1.0)])






Dropdown(description='Select F1 Winner: ', options=('hamilton', 'rosberg', 'alonso', 'kovalainen', 'raikkonen'…

Output()

In [154]:
# now I want to see the number of different winners per decade 

# display(drivers_df.head(5))
# display(races_df.head(5))
# display(circuits_df.head(5))
# display(results_df.head(5))

# ok, lets take the races and then see if we can do something with the series. 
races_df['datetime'] = pd.to_datetime(races_df['date'])
races_df.set_index('datetime', inplace=True)
races_df.sort_index(axis = 0, inplace=True) 
# isplay(races_df.head(5))

# now we need to go and find the winner for each one and add it.

cut_down_results = results_df[['raceId', 'driverId']].loc[results_df['positionText'] == '1']
# display(cut_down_results.head(5))

answer_df = pd.merge(races_df, cut_down_results, on='raceId')
answer_df = pd.merge(answer_df, drivers_df[['driverId', 'driverRef']], on='driverId')
answer_df.sort_values(by='date', inplace=True, ascending=True)
display(answer_df.head(10))
display(drivers_df.head(5))



,raceId,year,round,circuitId,name,date,time,url,driverId,driverRef
0,833,1950,1,9,British Grand Prix,1950-05-13,,http://en.wikipedia.org/wiki/1950_British_Gran...,642,farina
5,834,1950,2,6,Monaco Grand Prix,1950-05-21,,http://en.wikipedia.org/wiki/1950_Monaco_Grand...,579,fangio
29,835,1950,3,19,Indianapolis 500,1950-05-30,,http://en.wikipedia.org/wiki/1950_Indianapolis...,593,parsons
1,836,1950,4,66,Swiss Grand Prix,1950-06-04,,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...,642,farina
6,837,1950,5,13,Belgian Grand Prix,1950-06-18,,http://en.wikipedia.org/wiki/1950_Belgian_Gran...,579,fangio
7,838,1950,6,55,French Grand Prix,1950-07-02,,http://en.wikipedia.org/wiki/1950_French_Grand...,579,fangio
2,839,1950,7,14,Italian Grand Prix,1950-09-03,,http://en.wikipedia.org/wiki/1950_Italian_Gran...,642,farina
8,825,1951,1,66,Swiss Grand Prix,1951-05-27,,http://en.wikipedia.org/wiki/1951_Swiss_Grand_...,579,fangio
30,826,1951,2,19,Indianapolis 500,1951-05-30,,http://en.wikipedia.org/wiki/1951_Indianapolis...,766,wallard
3,827,1951,3,13,Belgian Grand Prix,1951-06-17,,http://en.wikipedia.org/wiki/1951_Belgian_Gran...,642,farina


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
